In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd
from scipy import signal
import pickle
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler,MaxAbsScaler
from sklearn.svm import SVR
import random
import tensorflow as tf
import cv2
from scipy import stats
import os


In [2]:
# Add tensorboard stuff
# %load_ext tensorboard
# import datetime

# !rm -rf ./logs/


In [3]:
#fix all seeds for random number generators
# Set seed value
seed_value = 9
import os
os.environ['PYTHONHASHSEED']=str(seed_value)# 2. Set `python` built-in pseudo-random generator at a fixed value
random.seed(seed_value)# 3. Set `numpy` pseudo-random generator at a fixed value
np.random.seed(seed_value)
tf.random.set_seed(seed_value)# 5. Configure a new global `tensorflow` session

# TODO need to add session thing for tensorflow

# Read Depth Camera

In [4]:
# For reading depth camera
def read_depth_camera(dcamera_path, show_video, nw_resize=1, nh_resize=1):
    video  = cv2.VideoCapture(dcamera_path)
    ret, frame = video.read()
    
    # Get total # of frame count 
    frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
        
    frame_height = int(frame.shape[0])
    frame_width = int(frame.shape[1])

    
    depth_frames = np.empty((frame_count, int(frame_height/nh_resize), int(frame_width/nw_resize)))
    depth_frames = np.empty((frame_count, int(frame_height/nh_resize), int(frame_width/nw_resize),3))
    count = 0
    while (video.isOpened()):
        ret, frame = video.read()
        
        if ret == True:
#             gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#             gray_frame = frame/np.maximum(np.max(frame),255) keep as int8 for memory savings
            gray_frame = frame
            gray_frame = cv2.resize(gray_frame, \
                                    (int(frame_width/nw_resize), int(frame_height/nh_resize)),\
                                    interpolation = cv2.INTER_NEAREST)

            depth_frames[count] = gray_frame
            if show_video == True:
                cv2.imshow("Depth", gray_frame)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
            count = count + 1
        else: 
            break
            

    video.release()
    #cv2.destroyAllWindows()
    return depth_frames

def read_output_data(qtm_file_data, theta):
    if theta=='x':
        qtm_data = pd.read_csv(qtm_file_data, usecols = ["Lean Left/Right Angle (deg)"])
    if theta=='y':
        qtm_data = pd.read_csv(qtm_file_data, usecols = ["Lean Forward/Backwards Angle (deg)"])
    if theta=='z':
        qtm_data = pd.read_csv(qtm_file_data, usecols = ["Torso Twist Angle (deg)"])
        
    
    return qtm_data


In [5]:
def play_video(read_frames):

    for i in range(len(read_frames)):
        # Capture frame-by-frame
        frame = read_frames[i]

        # Display the resulting frame
        plt.imshow(frame)
#         if cv2.waitKey(1) & 0xFF == ord('q'):
#             break
    #cv2.destroyAllWindows()

    return

In [6]:
def show_frame(read_frames, frame_range):
      # Capture frame-by-frame

    # Display the resulting frame
    for i in frame_range:
        fig = plt.figure(figsize=(6,6))        
        a = plt.imshow(read_frames[i])
        
    return

In [7]:
#fix this to concatenate everything
# n_test = 24
# date = '11_15_2020'
# test_str = '/test' + str(n_test)
# #data_dir = r"C:\Users\77bis\Box\CS598 - Final Project\Preliminary Data V4\Test_Subject_Leo" + test_str
# data_dir = "/home/asilador/CS598/CS598-FinalProject/Preliminary Data V4/Test_Subject_Leo" + test_str
# train_dcamera_path = data_dir + '/depth_processed_leo_test'+str(n_test)+'.avi'
# show_video = 0
# n_resize = 1
# x_train = read_depth_camera(train_dcamera_path, show_video, nw_resize=2, nh_resize=3)

n_test = (24,30,31,32,33,35)
nw_resize = 2
nh_resize = 2
xtemp = {}
show_video = 0

subj = ['leo','leo','leo','leo','leo','leo']
        
for i in range(len(n_test)):
    test_str = 'test' + str(n_test[i])
    
#     data_dir = os.path.join('/home/asilador/CS598/CS598-FinalProject/Preliminary Data V5','Test_Subject_'+subj[i],test_str)
    data_dir = os.path.join(r'C:\Users\Alex\Box\CS598 - Final Project\Preliminary Data V5','Test_Subject_'+subj[i],test_str)
#     train_dcamera_path = data_dir + '/depth_processed_leo_test'+str(n_test[i])+'.avi'
    train_dcamera_path = os.path.join(data_dir , 'depth_processed_'+subj[i]+'_test'+str(n_test[i])+'.avi')
    xtemp[i] = read_depth_camera(train_dcamera_path, show_video, nw_resize=nw_resize, nh_resize=nh_resize).astype('uint8')
    


In [8]:
tlen=0
for x in range(len(xtemp)):
    tlen+= xtemp[x].shape[0]
xrun_cum = 0
x_train = np.zeros((tlen,xtemp[0].shape[1],xtemp[0].shape[2],xtemp[0].shape[3]),dtype='uint8')
for i in range (len(xtemp)):
    xrun_n = len(xtemp[i])
    x_train[xrun_cum:xrun_cum+xrun_n,:,:,:] = xtemp[i][:xrun_n,:,:,:]
    xrun_cum += xrun_n

In [11]:
print(x_train.shape)
del xtemp

(153421, 60, 80, 3)


# read fcss data

In [12]:

n_test = (24,30,31,32,33,35)
date = ('11_15_2020','11_24_2020','11_24_2020','11_25_2020','11_25_2020','11_25_2020')
subj = ['leo','leo','leo','leo','leo','leo']
subjwgt = [67, 67, 67, 67, 67, 67]
subjht = [174, 174, 174, 174, 174, 174]
xfcss_gt = {}
yrun = 0
for i in range(len(n_test)):
    test_str = 'test' + str(n_test[i])
#     data_dir = os.path.join('/home/asilador/CS598/CS598-FinalProject/Preliminary Data V5','Test_Subject_'+subj[i],test_str)
    data_dir = os.path.join(r'C:\Users\Alex\Box\CS598 - Final Project\Preliminary Data V5','Test_Subject_'+subj[i],test_str)
    fcss_data_dir = os.path.join(data_dir , 'fcss_processed_'+subj[i]+'_' + test_str + '_' + date[i] + '.txt')
    xfcss_gttemp = pd.read_csv(fcss_data_dir)/subjwgt[i]*subjht[i]**2
    xfcss_gt[i]=xfcss_gttemp
#     xfcss_gt[i]['weight'] = subjwgt[i]# hacky but will work for now
#     xfcss_gt[i]['height'] = subjht[i]# hacky but will work for now
    if i==0:
        xfcss_train=xfcss_gttemp
    else:
        xfcss_train = pd.concat([xfcss_train,xfcss_gt[i]],axis=0)
del xfcss_gt

# Read Output Data


In [14]:

n_test = (24,30,31,32,33,35)
date = ('11_15_2020','11_24_2020','11_24_2020','11_25_2020','11_25_2020','11_25_2020')
subj = ['cz','cz','leo','leo','leo','leo','leo','leo','yc','yc']
y_gt = {}
yrun = 0
theta_interest = 'z'
for i in range(len(n_test)):
    test_str = 'test' + str(n_test[i])
#     data_dir = os.path.join('/home/asilador/CS598/CS598-FinalProject/Preliminary Data V5','Test_Subject_'+subj[i],test_str)
    data_dir = os.path.join(r'C:\Users\Alex\Box\CS598 - Final Project\Preliminary Data V5','Test_Subject_'+subj[i],test_str)
    qtm_file_data_dir = os.path.join(data_dir , 'qtm_processed_'+subj[i]+'_test' + str(n_test[i]) + '_' + date[i] + '.txt')
    y_gt[i] = read_output_data(qtm_file_data_dir,theta_interest).values



In [15]:
tlen=0
for x in range(len(y_gt)):
    tlen+= y_gt[x].shape[0]
yrun_cum = 0
y_train = np.zeros((tlen,1))
for i in range (len(y_gt)):
    yrun_n = len(y_gt[i])
    y_train[yrun_cum:yrun_cum+yrun_n] = y_gt[i][:]
    yrun_cum += yrun_n

In [16]:
# Saturate output
def saturate(theta, min_val, max_val):
    for i in range(len(theta)):
        if theta[i] < min_val:
            theta[i] = min_val
            continue
        if theta[i] > max_val:
            theta[i] = max_val
            continue
    return theta
            
min_val = -50
max_val = 50
    
y_train = saturate(y_train, min_val, max_val)

In [17]:
print(x_train.shape)
print(xfcss_train.shape)
print(y_train.shape)

(153421, 60, 80, 3)
(153421, 6)
(153421, 1)


In [18]:
nsamps = x_train.shape[0]
n80p = int(np.floor(nsamps*0.8))
rannums = np.array(random.sample(range(1,nsamps,1), n80p))
s_nfiles = np.arange(nsamps)
test_set = np.setdiff1d(s_nfiles,rannums)

#

In [34]:
Trainset = x_train[rannums,:]
Trainset2 = xfcss_train.values[rannums,:]
Testset = x_train[test_set,:]
Testset2 = xfcss_train.values[test_set,:]
# Trainy= y_gt[rannums,:]
# Testy = y_gt[test_set,:]
Trainy= y_train[rannums,:]
Testy = y_train[test_set,:]

In [20]:
# sc_X2 = StandardScaler()
# sc_y = StandardScaler()

sc_X2 = MaxAbsScaler()
sc_y = MaxAbsScaler()

In [21]:
Xtrainz = Trainset
Xtrainz2 = Trainset2
ytrainz = Trainy
X = Xtrainz
X2 = sc_X2.fit_transform(Xtrainz2)
y = sc_y.fit_transform(ytrainz)

In [22]:
#make validation data available to model.fit
Xvalid = Testset
Xvalid2 = sc_X2.transform(Testset2)
y_valid = Testy
y_valid = sc_y.transform(y_valid)


In [23]:
# clear up some used variables
del Trainset
del Trainset2
del Testset
del Testset2


# Regress using Neural Network

In [25]:
# Create Neural Netowrk

from tensorflow.keras.layers import Bidirectional, Conv2D, MaxPooling2D, Input, concatenate, AveragePooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dense, Activation, Dropout, Reshape, Permute, Flatten
from tensorflow.keras.models import Model
# dropout_rate = 0.2

model_start = Input(shape=(x_train.shape[1],x_train.shape[2],x_train.shape[3]))
model_start2 = Input(shape=(xfcss_train.shape[1],))
model_cnn = model_start
model_perc = model_start2

model_cnn = Conv2D(filters=8, kernel_size=(3, 3),padding='same',use_bias=False)(model_cnn)
model_cnn = BatchNormalization()(model_cnn)
model_cnn = Activation('relu')(model_cnn)
model_cnn = Conv2D(filters=8, kernel_size=(3, 3),padding='same',use_bias=False)(model_cnn)
model_cnn = BatchNormalization()(model_cnn)
model_cnn = Activation('relu')(model_cnn)
model_cnn = AveragePooling2D(pool_size=(2, 2))(model_cnn)

model_perc = Dense(100)(model_perc)
model_perc = Activation('relu')(model_perc)

model_cnn = Conv2D(filters=16, kernel_size=(3, 3),padding='same',use_bias=False)(model_cnn)
model_cnn = BatchNormalization()(model_cnn)
model_cnn = Activation('relu')(model_cnn)
model_cnn = Conv2D(filters=16, kernel_size=(3, 3),padding='same',use_bias=False)(model_cnn)
model_cnn = BatchNormalization()(model_cnn)
model_cnn = Activation('relu')(model_cnn)
model_cnn = AveragePooling2D(pool_size=(2, 2))(model_cnn)

model_perc = Dense(100)(model_perc)
model_perc = Activation('relu')(model_perc)

model_cnn = Conv2D(filters=32, kernel_size=(3, 3),padding='same',use_bias=False)(model_cnn)
model_cnn = BatchNormalization()(model_cnn)
model_cnn = Activation('relu')(model_cnn)
model_cnn = Conv2D(filters=32, kernel_size=(3, 3),padding='same',use_bias=False)(model_cnn)
model_cnn = BatchNormalization()(model_cnn)
model_cnn = Activation('relu')(model_cnn)
model_cnn = AveragePooling2D(pool_size=(2, 2))(model_cnn)

model_cnn = Conv2D(filters=64, kernel_size=(3, 3),padding='same',use_bias=False)(model_cnn)
model_cnn = BatchNormalization()(model_cnn)
model_cnn = Activation('relu')(model_cnn)
model_cnn = Conv2D(filters=64, kernel_size=(3, 3),padding='same',use_bias=False)(model_cnn)
model_cnn = BatchNormalization()(model_cnn)
model_cnn = Activation('relu')(model_cnn)
model_cnn = AveragePooling2D(pool_size=(2, 2))(model_cnn)

model_cnn = Flatten()(model_cnn)
# model_perc = Flatten()(model_perc)
# model_cnn = Activation('relu')(model_cnn)

model_cnn = Dense(100)(model_cnn)
model_cnn = Activation('relu')(model_cnn)
# model_cnn = Dropout(dropout_rate)(model_cnn)

model_comb = concatenate([model_cnn,model_perc],axis=-1)

model_comb = Dense(128)(model_comb)
# model_comb = BatchNormalization()(model_comb)
model_comb = Activation('relu')(model_comb)
# model_comb = Dropout(dropout_rate)(model_comb)

output = Dense(1)(model_comb)
output = Activation('linear', name='thetaz_out')(output)
model = Model(inputs=[model_start,model_start2],outputs=output)
model.compile(optimizer='adam',
              loss='mae',
              metrics=['mse','mae'])


# callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50,restore_best_weights=True) #Moving to 1000 patience. 
# callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=50,restore_best_weights=True) #Moving to 1000 patience.


In [26]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 60, 80, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 60, 80, 8)    216         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 60, 80, 8)    32          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 60, 80, 8)    0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [27]:
print(tf.__version__)

2.3.1


In [28]:
tf.config.list_physical_devices('GPU') 

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [29]:
from tensorflow.keras import layers
with tf.device('/device:CPU:0'):
    data_augmentation = tf.keras.Sequential([
        layers.experimental.preprocessing.RandomRotation(0.05),
        layers.experimental.preprocessing.RandomZoom(height_factor = (-0.2,0.2),
                                               width_factor = (-0.2,0.2), 
                                               fill_mode = 'constant'),
        ], 
        name='data_augmentation')

# create data generator
def get_generator_cyclic(features1, features2, labels, batch_size=256):
    while True:
        for n in range(int(len(features1)/batch_size)):
            X = features1[n*batch_size: (n+1)*batch_size]
            with tf.device('/device:CPU:0'): #to prevent hogging limited gpu space
                augmented_images = data_augmentation(X)
                Xnew =  tf.cast(augmented_images,tf.float64)/255
            yield [Xnew, features2[n*batch_size: (n+1)*batch_size], labels[n*batch_size: (n+1)*batch_size]]
        permuted = np.random.permutation(len(features1))
        features1 = features1[permuted]
        features2 = features2[permuted]
        labels = labels[permuted]


In [30]:
def scheduler(epoch, lr):
    if e%300 < 200: #very hacky
        return 0.001
    else:
        if e%25==0 & batches==0:
            return lr * 0.99
        else:
            return lr
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [31]:
from datetime import datetime
batch_size = 128
batch_mult = 4
readin = int(batch_size*batch_mult)
epochs = int(1200)
mae_best = 10000
training_generator = get_generator_cyclic(X,X2,y,readin)
# test_generator = get_generator_cyclic(Xvalid,Xvalid2,y_valid,readin)
for e in range(epochs):
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    print('Epoch', e,'Time: ', current_time)
    batches = 0
    while batches< len(X)/readin:
        Xtrain_1, Xtrain_2, ytrain_1 = next(training_generator)
#         Xtest_1, Xtest_2, ytest_1 =next(test_generator)
#         model.fit([Xtrain_1, Xtrain_2], ytrain_1, callbacks = [callback],validation_data = ([Xtest_1,Xtest_2], ytest_1),batch_size=batch_size,verbose = 0)
        model.fit([Xtrain_1, Xtrain_2], ytrain_1, callbacks = [callback],batch_size=batch_size,verbose = 0)
        batches += 1
    
    #calculates and prints the running validation once per epoch
    losssc, msesc, maesc = model.evaluate([Xvalid/255.,Xvalid2],y_valid,verbose=0)
    mae = sc_y.inverse_transform(np.array(maesc).reshape(1,-1))[0][0]
    if mae<mae_best:
        modelbest = model
        mae_best = mae
    print('Mean absolute error at {:4.0f} is: {:4.2f}'.format(e,mae))
modelbest.save('cnn3v28')    

Epoch 0 Time:  21:22:42
Mean absolute error at    0 is: 6.80
Epoch 1 Time:  21:24:03
Mean absolute error at    1 is: 5.46
Epoch 2 Time:  21:25:21
Mean absolute error at    2 is: 6.93
Epoch 3 Time:  21:26:38
Mean absolute error at    3 is: 4.85
Epoch 4 Time:  21:27:55
Mean absolute error at    4 is: 5.11
Epoch 5 Time:  21:29:12
Mean absolute error at    5 is: 6.34
Epoch 6 Time:  21:30:28
Mean absolute error at    6 is: 5.09
Epoch 7 Time:  21:31:45
Mean absolute error at    7 is: 4.47
Epoch 8 Time:  21:33:01
Mean absolute error at    8 is: 4.17
Epoch 9 Time:  21:34:18
Mean absolute error at    9 is: 4.13
Epoch 10 Time:  21:35:35
Mean absolute error at   10 is: 4.39
Epoch 11 Time:  21:36:52
Mean absolute error at   11 is: 3.84
Epoch 12 Time:  21:38:09
Mean absolute error at   12 is: 4.21
Epoch 13 Time:  21:39:26
Mean absolute error at   13 is: 4.05
Epoch 14 Time:  21:40:42
Mean absolute error at   14 is: 3.92
Epoch 15 Time:  21:41:59
Mean absolute error at   15 is: 3.64
Epoch 16 Time:  21

Mean absolute error at  132 is: 2.45
Epoch 133 Time:  00:13:07
Mean absolute error at  133 is: 2.54
Epoch 134 Time:  00:14:25
Mean absolute error at  134 is: 2.44
Epoch 135 Time:  00:15:41
Mean absolute error at  135 is: 2.47
Epoch 136 Time:  00:16:57
Mean absolute error at  136 is: 2.42
Epoch 137 Time:  00:18:14
Mean absolute error at  137 is: 2.37
Epoch 138 Time:  00:19:31
Mean absolute error at  138 is: 2.42
Epoch 139 Time:  00:20:47
Mean absolute error at  139 is: 2.43
Epoch 140 Time:  00:22:04
Mean absolute error at  140 is: 2.47
Epoch 141 Time:  00:23:20
Mean absolute error at  141 is: 2.45
Epoch 142 Time:  00:24:37
Mean absolute error at  142 is: 2.43
Epoch 143 Time:  00:25:53
Mean absolute error at  143 is: 2.38
Epoch 144 Time:  00:27:10
Mean absolute error at  144 is: 2.58
Epoch 145 Time:  00:28:26
Mean absolute error at  145 is: 2.38
Epoch 146 Time:  00:29:43
Mean absolute error at  146 is: 2.41
Epoch 147 Time:  00:31:00
Mean absolute error at  147 is: 2.38
Epoch 148 Time:  0

Mean absolute error at  263 is: 1.97
Epoch 264 Time:  03:00:21
Mean absolute error at  264 is: 1.97
Epoch 265 Time:  03:01:38
Mean absolute error at  265 is: 1.97
Epoch 266 Time:  03:02:55
Mean absolute error at  266 is: 1.97
Epoch 267 Time:  03:04:11
Mean absolute error at  267 is: 1.97
Epoch 268 Time:  03:05:28
Mean absolute error at  268 is: 1.97
Epoch 269 Time:  03:06:44
Mean absolute error at  269 is: 1.97
Epoch 270 Time:  03:08:01
Mean absolute error at  270 is: 1.97
Epoch 271 Time:  03:09:17
Mean absolute error at  271 is: 1.97
Epoch 272 Time:  03:10:34
Mean absolute error at  272 is: 1.97
Epoch 273 Time:  03:11:50
Mean absolute error at  273 is: 1.97
Epoch 274 Time:  03:13:07
Mean absolute error at  274 is: 1.97
Epoch 275 Time:  03:14:23
Mean absolute error at  275 is: 1.97
Epoch 276 Time:  03:15:40
Mean absolute error at  276 is: 1.97
Epoch 277 Time:  03:16:56
Mean absolute error at  277 is: 1.97
Epoch 278 Time:  03:18:13
Mean absolute error at  278 is: 1.97
Epoch 279 Time:  0

Mean absolute error at  394 is: 2.13
Epoch 395 Time:  05:47:36
Mean absolute error at  395 is: 2.16
Epoch 396 Time:  05:48:53
Mean absolute error at  396 is: 2.18
Epoch 397 Time:  05:50:09
Mean absolute error at  397 is: 2.24
Epoch 398 Time:  05:51:26
Mean absolute error at  398 is: 2.13
Epoch 399 Time:  05:52:43
Mean absolute error at  399 is: 2.25
Epoch 400 Time:  05:53:59
Mean absolute error at  400 is: 2.18
Epoch 401 Time:  05:55:16
Mean absolute error at  401 is: 2.26
Epoch 402 Time:  05:56:32
Mean absolute error at  402 is: 2.09
Epoch 403 Time:  05:57:49
Mean absolute error at  403 is: 2.10
Epoch 404 Time:  05:59:05
Mean absolute error at  404 is: 2.14
Epoch 405 Time:  06:00:22
Mean absolute error at  405 is: 2.15
Epoch 406 Time:  06:01:39
Mean absolute error at  406 is: 2.22
Epoch 407 Time:  06:02:55
Mean absolute error at  407 is: 2.23
Epoch 408 Time:  06:04:12
Mean absolute error at  408 is: 2.10
Epoch 409 Time:  06:05:28
Mean absolute error at  409 is: 2.10
Epoch 410 Time:  0

Mean absolute error at  525 is: 1.81
Epoch 526 Time:  08:35:11
Mean absolute error at  526 is: 1.81
Epoch 527 Time:  08:36:27
Mean absolute error at  527 is: 1.81
Epoch 528 Time:  08:37:44
Mean absolute error at  528 is: 1.81
Epoch 529 Time:  08:39:00
Mean absolute error at  529 is: 1.81
Epoch 530 Time:  08:40:17
Mean absolute error at  530 is: 1.81
Epoch 531 Time:  08:41:33
Mean absolute error at  531 is: 1.81
Epoch 532 Time:  08:42:50
Mean absolute error at  532 is: 1.81
Epoch 533 Time:  08:44:06
Mean absolute error at  533 is: 1.81
Epoch 534 Time:  08:45:23
Mean absolute error at  534 is: 1.81
Epoch 535 Time:  08:46:39
Mean absolute error at  535 is: 1.81
Epoch 536 Time:  08:47:56
Mean absolute error at  536 is: 1.81
Epoch 537 Time:  08:49:12
Mean absolute error at  537 is: 1.81
Epoch 538 Time:  08:50:29
Mean absolute error at  538 is: 1.81
Epoch 539 Time:  08:51:45
Mean absolute error at  539 is: 1.81
Epoch 540 Time:  08:53:02
Mean absolute error at  540 is: 1.81
Epoch 541 Time:  0

Mean absolute error at  656 is: 2.05
Epoch 657 Time:  11:22:20
Mean absolute error at  657 is: 2.07
Epoch 658 Time:  11:23:36
Mean absolute error at  658 is: 2.03
Epoch 659 Time:  11:24:53
Mean absolute error at  659 is: 2.12
Epoch 660 Time:  11:26:09
Mean absolute error at  660 is: 2.05
Epoch 661 Time:  11:27:26
Mean absolute error at  661 is: 2.03
Epoch 662 Time:  11:28:42
Mean absolute error at  662 is: 2.08
Epoch 663 Time:  11:29:59
Mean absolute error at  663 is: 2.06
Epoch 664 Time:  11:31:16
Mean absolute error at  664 is: 2.04
Epoch 665 Time:  11:32:32
Mean absolute error at  665 is: 2.00
Epoch 666 Time:  11:33:49
Mean absolute error at  666 is: 2.01
Epoch 667 Time:  11:35:05
Mean absolute error at  667 is: 2.07
Epoch 668 Time:  11:36:22
Mean absolute error at  668 is: 2.05
Epoch 669 Time:  11:37:38
Mean absolute error at  669 is: 2.04
Epoch 670 Time:  11:38:55
Mean absolute error at  670 is: 1.99
Epoch 671 Time:  11:40:11
Mean absolute error at  671 is: 2.02
Epoch 672 Time:  1

Mean absolute error at  787 is: 1.99
Epoch 788 Time:  14:10:36
Mean absolute error at  788 is: 1.95
Epoch 789 Time:  14:11:53
Mean absolute error at  789 is: 2.00
Epoch 790 Time:  14:13:09
Mean absolute error at  790 is: 2.07
Epoch 791 Time:  14:14:26
Mean absolute error at  791 is: 2.02
Epoch 792 Time:  14:15:42
Mean absolute error at  792 is: 2.04
Epoch 793 Time:  14:16:58
Mean absolute error at  793 is: 2.00
Epoch 794 Time:  14:18:15
Mean absolute error at  794 is: 1.95
Epoch 795 Time:  14:19:31
Mean absolute error at  795 is: 2.01
Epoch 796 Time:  14:20:47
Mean absolute error at  796 is: 2.00
Epoch 797 Time:  14:22:04
Mean absolute error at  797 is: 1.97
Epoch 798 Time:  14:23:20
Mean absolute error at  798 is: 1.94
Epoch 799 Time:  14:24:36
Mean absolute error at  799 is: 1.94
Epoch 800 Time:  14:25:53
Mean absolute error at  800 is: 1.79
Epoch 801 Time:  14:27:10
Mean absolute error at  801 is: 1.78
Epoch 802 Time:  14:28:26
Mean absolute error at  802 is: 1.78
Epoch 803 Time:  1

Mean absolute error at  918 is: 1.98
Epoch 919 Time:  16:57:29
Mean absolute error at  919 is: 1.95
Epoch 920 Time:  16:58:46
Mean absolute error at  920 is: 1.99
Epoch 921 Time:  17:00:02
Mean absolute error at  921 is: 1.97
Epoch 922 Time:  17:01:19
Mean absolute error at  922 is: 1.96
Epoch 923 Time:  17:02:35
Mean absolute error at  923 is: 1.92
Epoch 924 Time:  17:03:52
Mean absolute error at  924 is: 1.98
Epoch 925 Time:  17:05:08
Mean absolute error at  925 is: 1.94
Epoch 926 Time:  17:06:24
Mean absolute error at  926 is: 1.97
Epoch 927 Time:  17:07:41
Mean absolute error at  927 is: 2.00
Epoch 928 Time:  17:08:58
Mean absolute error at  928 is: 1.98
Epoch 929 Time:  17:10:14
Mean absolute error at  929 is: 1.98
Epoch 930 Time:  17:11:31
Mean absolute error at  930 is: 1.95
Epoch 931 Time:  17:12:47
Mean absolute error at  931 is: 2.05
Epoch 932 Time:  17:14:04
Mean absolute error at  932 is: 1.97
Epoch 933 Time:  17:15:20
Mean absolute error at  933 is: 1.90
Epoch 934 Time:  1

Mean absolute error at 1048 is: 1.96
Epoch 1049 Time:  19:44:03
Mean absolute error at 1049 is: 2.09
Epoch 1050 Time:  19:45:20
Mean absolute error at 1050 is: 1.96
Epoch 1051 Time:  19:46:36
Mean absolute error at 1051 is: 1.94
Epoch 1052 Time:  19:47:53
Mean absolute error at 1052 is: 1.95
Epoch 1053 Time:  19:49:09
Mean absolute error at 1053 is: 1.93
Epoch 1054 Time:  19:50:26
Mean absolute error at 1054 is: 1.95
Epoch 1055 Time:  19:51:43
Mean absolute error at 1055 is: 1.93
Epoch 1056 Time:  19:52:59
Mean absolute error at 1056 is: 1.95
Epoch 1057 Time:  19:54:15
Mean absolute error at 1057 is: 1.93
Epoch 1058 Time:  19:55:32
Mean absolute error at 1058 is: 1.93
Epoch 1059 Time:  19:56:49
Mean absolute error at 1059 is: 1.94
Epoch 1060 Time:  19:58:05
Mean absolute error at 1060 is: 1.93
Epoch 1061 Time:  19:59:22
Mean absolute error at 1061 is: 1.91
Epoch 1062 Time:  20:00:39
Mean absolute error at 1062 is: 1.94
Epoch 1063 Time:  20:01:55
Mean absolute error at 1063 is: 1.93
Epo

Mean absolute error at 1177 is: 1.71
Epoch 1178 Time:  22:28:48
Mean absolute error at 1178 is: 1.71
Epoch 1179 Time:  22:30:05
Mean absolute error at 1179 is: 1.70
Epoch 1180 Time:  22:31:21
Mean absolute error at 1180 is: 1.70
Epoch 1181 Time:  22:32:38
Mean absolute error at 1181 is: 1.71
Epoch 1182 Time:  22:33:55
Mean absolute error at 1182 is: 1.71
Epoch 1183 Time:  22:35:11
Mean absolute error at 1183 is: 1.70
Epoch 1184 Time:  22:36:28
Mean absolute error at 1184 is: 1.70
Epoch 1185 Time:  22:37:44
Mean absolute error at 1185 is: 1.71
Epoch 1186 Time:  22:39:01
Mean absolute error at 1186 is: 1.71
Epoch 1187 Time:  22:40:17
Mean absolute error at 1187 is: 1.71
Epoch 1188 Time:  22:41:34
Mean absolute error at 1188 is: 1.71
Epoch 1189 Time:  22:42:51
Mean absolute error at 1189 is: 1.70
Epoch 1190 Time:  22:44:07
Mean absolute error at 1190 is: 1.70
Epoch 1191 Time:  22:45:24
Mean absolute error at 1191 is: 1.71
Epoch 1192 Time:  22:46:41
Mean absolute error at 1192 is: 1.71
Epo

In [32]:
save_notebookparams = 1
if save_notebookparams:
    pkl_filename = "depthforcemodelparam_cnn3v28_pb.pkl"
    randata = {}
    randata['nsamps']=nsamps
    randata['n80p']=n80p
    randata['rannums']=rannums
    randata['test_set']=test_set
#     modelhistory = history.history
    
    
    with open(pkl_filename, 'wb') as file:
#         pickle.dump([randata,sc_y,sc_X2,modelhistory], file)
          pickle.dump([randata,sc_y,sc_X2], file)
        
#     !mkdir -p saved_model
#     model.save('saved_model/cnn3v22')